In [2]:
import pandas as pd
import numpy as np

In [3]:
intrst_cols=['Make','Model','Year','Engine HP','Engine Cylinders','Transmission Type','Vehicle Style','highway MPG','city mpg']
dataset = pd.read_csv('data.csv')
dataset = pd.concat([dataset[intrst_cols], dataset['MSRP']],axis=1)
dataset.columns = dataset.columns.str.replace(' ', '_').str.lower()
dataset.rename(columns={'msrp':'price'},inplace=True)
dataset.columns[dataset.isnull().any(axis=0)]

Index(['engine_hp', 'engine_cylinders'], dtype='object')

In [4]:
dataset.fillna(value={'engine_hp':0, 'engine_cylinders':0},inplace=True)

In [5]:
dataset.columns[dataset.isnull().any(axis=0)]

Index([], dtype='object')

## Q1

Automatic

In [6]:
dataset.transmission_type.value_counts()

transmission_type
AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: count, dtype: int64

## Q2
Highway mpg and city mpg

In [7]:
num_cols = ['year','engine_hp','engine_cylinders','highway_mpg','city_mpg']
dataset[num_cols].corr()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg
year,1.000000,0.338714,-0.040708,0.258240,0.198171
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000


In [8]:
avg_price = dataset.price.mean()
dataset['above_average'] = (dataset['price']>avg_price).astype(int)
del dataset['price']

In [9]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(dataset, test_size=0.2, random_state=42)

df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)



## Q3
transmission_type

In [10]:
from sklearn.metrics import mutual_info_score

mis = dict()

for cat_var in ['make','model','transmission_type','vehicle_style']:
    mis[cat_var]=round(mutual_info_score(df_train['above_average'], df_train[cat_var]),2)

y_train, y_test, y_val = df_train.above_average, df_test.above_average, df_val.above_average 

del df_train['above_average']
del df_test['above_average']
del df_val['above_average']

mis

{'make': 0.24, 'model': 0.46, 'transmission_type': 0.02, 'vehicle_style': 0.08}

## Q4
0.94

In [11]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')

X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [12]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

model.fit(X_train,y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [13]:
y_pred = model.predict_proba(X_val)[:,1]
abv_avg = (y_pred>=0.5)
df_pred = pd.DataFrame()
df_pred['probability'] = model.predict_proba(X_val)[:,1]
df_pred['prediction'] = abv_avg.astype(int)
df_pred['actual'] = y_val
df_pred['correct'] = df_pred.prediction == df_pred.actual
round(df_pred.correct.mean(),2)

0.11

In [14]:
from sklearn.metrics import accuracy_score
score = model.score(X_val, y_val)
round(score,2)

0.95

## Q5
city mpg

In [15]:


acc_dict = dict()

listofstuff = [['year'],['engine_hp'],['transmission_type'],['city_mpg'],['model'],['engine_cylinders'],['vehicle_style'],['highway_mpg'],['city_mpg']] 

for target_list in listofstuff:
    df_train_t = df_train.drop(target_list,axis=1)
    df_val_t = df_val.drop(target_list,axis=1)
    train_dictt = df_train_t.to_dict(orient='records')
    dvt = DictVectorizer(sparse=False)
    X_traint = dvt.fit_transform(train_dictt)
    val_dict = df_val_t.to_dict(orient='records')
    X_valt = dvt.transform(val_dict)
    modeltest = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    modeltest.fit(X_traint,y_train)
    scorek = modeltest.score(X_valt, y_val)
    acc_dict[target_list[0]]=scorek

acc_dict["normal"]=scorek
diff_dict = dict()
for n in acc_dict.keys():
    diff_dict[n] = abs(scorek - acc_dict[n])

diff_dict

{'year': 0.0016785564414604215,
 'engine_hp': 0.022660511959714635,
 'transmission_type': 0.0012589173310951773,
 'city_mpg': 0.0,
 'model': 0.026856903063365523,
 'engine_cylinders': 0.0008392782207301552,
 'vehicle_style': 0.003776751993285754,
 'highway_mpg': 0.004196391103650776,
 'normal': 0.0}

## Q6


In [16]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

datasetL = pd.read_csv('data.csv')
datasetL = pd.concat([datasetL[intrst_cols], datasetL['MSRP']],axis=1)
datasetL.columns = datasetL.columns.str.replace(' ', '_').str.lower()
datasetL.rename(columns={'msrp':'price'},inplace=True)
datasetL.columns[datasetL.isnull().any(axis=0)]
datasetL.fillna(value={'engine_hp':0, 'engine_cylinders':0},inplace=True)
datasetL['price'] = np.log(datasetL['price'])

In [17]:
df_full_trainL, df_testL = train_test_split(datasetL, test_size=0.2, random_state=42)

df_trainL, df_valL = train_test_split(df_full_trainL, test_size=0.25, random_state=42)

y_trainL, y_testL, y_valL = df_trainL.price, df_testL.price, df_valL.price 

del df_trainL['price']
del df_testL['price']
del df_valL['price']

In [21]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=True)

train_dictL = df_trainL.to_dict(orient='records')

X_trainL = dv.fit_transform(train_dictL)

val_dictL = df_valL.to_dict(orient='records')
X_valL = dv.transform(val_dictL)

In [25]:
from sklearn.metrics import mean_squared_error
alphas = [0, 0.01, 0.1, 1, 10]
rmse_scores = dict()
for alph in alphas:
    ridge = linear_model.Ridge(alpha=alph, solver='sag', random_state=42)
    ridge.fit(X_trainL, y_trainL)
    y_predL = ridge.predict(X_valL)
    
    # Calculate RMSE (Root Mean Squared Error)
    rmse = mean_squared_error(y_valL, y_predL,squared=False)
    
    # Round RMSE to 3 decimal digits
    rmse = round(rmse, 3)
    
    rmse_scores[alph] = rmse
